In [ ]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/mnt/snakemake', '/mnt/bfq-analysis/GCF-2021-832_211118/src/gcf-workflows/singlecell/rules/quant/scripts']); import pickle; snakemake = pickle.loads(b'\x80\x03csnakemake.script\nSnakemake\nq\x00)\x81q\x01}q\x02(X\x05\x00\x00\x00inputq\x03csnakemake.io\nInputFiles\nq\x04)\x81q\x05XE\x00\x00\x00data/tmp/singlecell/quant/aggregate/cellranger/all_samples_merge.h5adq\x06a}q\x07(X\x06\x00\x00\x00_namesq\x08}q\tX\x12\x00\x00\x00_allowed_overridesq\n]q\x0b(X\x05\x00\x00\x00indexq\x0cX\x04\x00\x00\x00sortq\reh\x0ccfunctools\npartial\nq\x0ecbuiltins\ngetattr\nq\x0fcsnakemake.io\nNamedlist\nq\x10X\x0f\x00\x00\x00_used_attributeq\x11\x86q\x12Rq\x13\x85q\x14Rq\x15(h\x13)}q\x16X\x05\x00\x00\x00_nameq\x17h\x0csNtq\x18bh\rh\x0eh\x13\x85q\x19Rq\x1a(h\x13)}q\x1bh\x17h\rsNtq\x1cbubX\x06\x00\x00\x00outputq\x1dcsnakemake.io\nOutputFiles\nq\x1e)\x81q\x1fXS\x00\x00\x00data/tmp/singlecell/quant/aggregate/cellranger/scanpy/all_samples_preprocessed.h5adq a}q!(h\x08}q"X\x0c\x00\x00\x00preprocessedq#K\x00N\x86q$sh\n]q%(h\x0ch\reh\x0ch\x0eh\x13\x85q&Rq\'(h\x13)}q(h\x17h\x0csNtq)bh\rh\x0eh\x13\x85q*Rq+(h\x13)}q,h\x17h\rsNtq-bh#h ubX\x06\x00\x00\x00paramsq.csnakemake.io\nParams\nq/)\x81q0}q1(h\x08}q2h\n]q3(h\x0ch\reh\x0ch\x0eh\x13\x85q4Rq5(h\x13)}q6h\x17h\x0csNtq7bh\rh\x0eh\x13\x85q8Rq9(h\x13)}q:h\x17h\rsNtq;bubX\t\x00\x00\x00wildcardsq<csnakemake.io\nWildcards\nq=)\x81q>X\x0b\x00\x00\x00all_samplesq?a}q@(h\x08}qAX\x07\x00\x00\x00aggr_idqBK\x00N\x86qCsh\n]qD(h\x0ch\reh\x0ch\x0eh\x13\x85qERqF(h\x13)}qGh\x17h\x0csNtqHbh\rh\x0eh\x13\x85qIRqJ(h\x13)}qKh\x17h\rsNtqLbX\x07\x00\x00\x00aggr_idqMh?ubX\x07\x00\x00\x00threadsqNK\x18X\t\x00\x00\x00resourcesqOcsnakemake.io\nResources\nqP)\x81qQ(K\x18K\x01e}qR(h\x08}qS(X\x06\x00\x00\x00_coresqTK\x00N\x86qUX\x06\x00\x00\x00_nodesqVK\x01N\x86qWuh\n]qX(h\x0ch\reh\x0ch\x0eh\x13\x85qYRqZ(h\x13)}q[h\x17h\x0csNtq\\bh\rh\x0eh\x13\x85q]Rq^(h\x13)}q_h\x17h\rsNtq`bhTK\x18hVK\x01ubX\x03\x00\x00\x00logqacsnakemake.io\nLog\nqb)\x81qcXM\x00\x00\x00data/tmp/singlecell/quant/aggregate/cellranger/notebooks/all_samples_pp.ipynbqda}qe(h\x08}qfX\x08\x00\x00\x00notebookqgK\x00N\x86qhsh\n]qi(h\x0ch\reh\x0ch\x0eh\x13\x85qjRqk(h\x13)}qlh\x17h\x0csNtqmbh\rh\x0eh\x13\x85qnRqo(h\x13)}qph\x17h\rsNtqqbhghdubX\x06\x00\x00\x00configqr}qs(X\n\x00\x00\x00project_idqt]quX\x0c\x00\x00\x00GCF-2021-832qvaX\x08\x00\x00\x00organismqwX\x0c\x00\x00\x00mus_musculusqxX\n\x00\x00\x00libprepkitqyXB\x00\x00\x0010X Genomics Chromium Single Cell 3p GEM Library & Gel Bead Kit v3qzX\r\x00\x00\x00read_geometryq{]q|(K\x1cK8eX\x07\x00\x00\x00machineq}X\x0b\x00\x00\x00NextSeq 500q~X\x05\x00\x00\x00quantq\x7f}q\x80(X\x05\x00\x00\x00batchq\x81}q\x82X\x06\x00\x00\x00methodq\x83X\x04\x00\x00\x00skipq\x84sX\n\x00\x00\x00cellrangerq\x85}q\x86(X\t\x00\x00\x00chemistryq\x87X\x06\x00\x00\x00SC3Pv3q\x88X\t\x00\x00\x00whitelistq\x89X\x14\x00\x00\x003M-february-2018.txtq\x8aX\x05\x00\x00\x00mkgtfq\x8b}q\x8cX\x0c\x00\x00\x00gene_biotypeq\x8d]q\x8e(X\x0e\x00\x00\x00protein_codingq\x8fX\x07\x00\x00\x00lincRNAq\x90esX\t\x00\x00\x00reanalyzeq\x91}q\x92X\x04\x00\x00\x00skipq\x93\x88suX\x08\x00\x00\x00starsoloq\x94}q\x95(X\t\x00\x00\x00whitelistq\x96X\x14\x00\x00\x003M-february-2018.txtq\x97X\x06\x00\x00\x00cb_lenq\x98K\x10X\x07\x00\x00\x00umi_lenq\x99K\x0cX\t\x00\x00\x00umi_startq\x9aK\x11uX\x06\x00\x00\x00alevinq\x9b}q\x9cX\t\x00\x00\x00chemistryq\x9dX\n\x00\x00\x00chromiumV3q\x9esX\t\x00\x00\x00umi_toolsq\x9f}q\xa0X\t\x00\x00\x00chemistryq\xa1X\x1c\x00\x00\x00CCCCCCCCCCCCCCCCNNNNNNNNNNNNq\xa2sX\x06\x00\x00\x00methodq\xa3X\n\x00\x00\x00cellrangerq\xa4X\t\x00\x00\x00aggregateq\xa5}q\xa6(X\x04\x00\x00\x00skipq\xa7\x89X\x06\x00\x00\x00methodq\xa8X\n\x00\x00\x00cellrangerq\xa9X\x04\x00\x00\x00normq\xaaX\x06\x00\x00\x00mappedq\xabX\x07\x00\x00\x00groupbyq\xacX\x0b\x00\x00\x00all_samplesq\xaduuX\t\x00\x00\x00fastq_dirq\xaeX\x0e\x00\x00\x00data/raw/fastqq\xafX\x07\x00\x00\x00samplesq\xb0}q\xb1(X\x01\x00\x00\x001q\xb2}q\xb3(X\x02\x00\x00\x00R1q\xb4X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/1/1_S1_L001_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/1/1_S1_L002_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/1/1_S1_L003_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/1/1_S1_L004_R1_001.fastq.gzq\xb5X\x02\x00\x00\x00R2q\xb6X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/1/1_S1_L001_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/1/1_S1_L002_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/1/1_S1_L003_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/1/1_S1_L004_R2_001.fastq.gzq\xb7X\n\x00\x00\x00paired_endq\xb8K\x01X\t\x00\x00\x00Sample_IDq\xb9h\xb2X\x0b\x00\x00\x00External_IDq\xbaX\x00\x00\x00\x00q\xbbX\n\x00\x00\x00Project_IDq\xbcX\x0c\x00\x00\x00GCF-2021-832q\xbdX\x08\x00\x00\x00Organismq\xbeh\xbbX\x10\x00\x00\x00Sample_Biosourceq\xbfh\xbbX[\x00\x00\x00Index1_p7 (If dual indexed libraries are submitted indicate what index sequence is used P7)q\xc0X\x08\x00\x00\x00SI-GA-B9q\xc1XP\x00\x00\x00Index2_p5 (If libraries are submitted  indicate what index sequence is used P75)q\xc2X\x08\x00\x00\x00SI-GA-B9q\xc3X\x0c\x00\x00\x00Sample_Groupq\xc4h\xbbX\x10\x00\x00\x00Customer_Commentq\xc5h\xbbX\n\x00\x00\x00Project IDq\xc6X\x0c\x00\x00\x00GCF-2021-832q\xc7X\x03\x00\x00\x00RINq\xc8h\xbbX\x07\x00\x00\x00SpikeInq\xc9h\xbbX\x0f\x00\x00\x00Fragment_Lengthq\xcaG@~\x00\x00\x00\x00\x00\x00X\x0b\x00\x00\x00Fragment_SDq\xcbh\xbbX\r\x00\x00\x00Concentrationq\xcch\xbbX\x07\x00\x00\x00260/280q\xcdh\xbbX\x07\x00\x00\x00260/230q\xceh\xbbX\x0b\x00\x00\x00Lab_Commentq\xcfh\xbbuX\x01\x00\x00\x002q\xd0}q\xd1(X\x02\x00\x00\x00R1q\xd2X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/2/2_S2_L001_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/2/2_S2_L002_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/2/2_S2_L003_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/2/2_S2_L004_R1_001.fastq.gzq\xd3X\x02\x00\x00\x00R2q\xd4X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/2/2_S2_L001_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/2/2_S2_L002_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/2/2_S2_L003_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/2/2_S2_L004_R2_001.fastq.gzq\xd5X\n\x00\x00\x00paired_endq\xd6K\x01X\t\x00\x00\x00Sample_IDq\xd7h\xd0X\x0b\x00\x00\x00External_IDq\xd8h\xbbX\n\x00\x00\x00Project_IDq\xd9X\x0c\x00\x00\x00GCF-2021-832q\xdaX\x08\x00\x00\x00Organismq\xdbh\xbbX\x10\x00\x00\x00Sample_Biosourceq\xdch\xbbX[\x00\x00\x00Index1_p7 (If dual indexed libraries are submitted indicate what index sequence is used P7)q\xddX\t\x00\x00\x00SI-GA-B10q\xdeXP\x00\x00\x00Index2_p5 (If libraries are submitted  indicate what index sequence is used P75)q\xdfX\t\x00\x00\x00SI-GA-B10q\xe0X\x0c\x00\x00\x00Sample_Groupq\xe1h\xbbX\x10\x00\x00\x00Customer_Commentq\xe2h\xbbX\n\x00\x00\x00Project IDq\xe3X\x0c\x00\x00\x00GCF-2021-832q\xe4X\x03\x00\x00\x00RINq\xe5h\xbbX\x07\x00\x00\x00SpikeInq\xe6h\xbbX\x0f\x00\x00\x00Fragment_Lengthq\xe7G@~\x00\x00\x00\x00\x00\x00X\x0b\x00\x00\x00Fragment_SDq\xe8h\xbbX\r\x00\x00\x00Concentrationq\xe9h\xbbX\x07\x00\x00\x00260/280q\xeah\xbbX\x07\x00\x00\x00260/230q\xebh\xbbX\x0b\x00\x00\x00Lab_Commentq\xech\xbbuX\x01\x00\x00\x003q\xed}q\xee(X\x02\x00\x00\x00R1q\xefX#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/3/3_S3_L001_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/3/3_S3_L002_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/3/3_S3_L003_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/3/3_S3_L004_R1_001.fastq.gzq\xf0X\x02\x00\x00\x00R2q\xf1X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/3/3_S3_L001_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/3/3_S3_L002_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/3/3_S3_L003_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/3/3_S3_L004_R2_001.fastq.gzq\xf2X\n\x00\x00\x00paired_endq\xf3K\x01X\t\x00\x00\x00Sample_IDq\xf4h\xedX\x0b\x00\x00\x00External_IDq\xf5h\xbbX\n\x00\x00\x00Project_IDq\xf6X\x0c\x00\x00\x00GCF-2021-832q\xf7X\x08\x00\x00\x00Organismq\xf8h\xbbX\x10\x00\x00\x00Sample_Biosourceq\xf9h\xbbX[\x00\x00\x00Index1_p7 (If dual indexed libraries are submitted indicate what index sequence is used P7)q\xfaX\t\x00\x00\x00SI-GA-B11q\xfbXP\x00\x00\x00Index2_p5 (If libraries are submitted  indicate what index sequence is used P75)q\xfcX\t\x00\x00\x00SI-GA-B11q\xfdX\x0c\x00\x00\x00Sample_Groupq\xfeh\xbbX\x10\x00\x00\x00Customer_Commentq\xffh\xbbX\n\x00\x00\x00Project IDr\x00\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832r\x01\x01\x00\x00X\x03\x00\x00\x00RINr\x02\x01\x00\x00h\xbbX\x07\x00\x00\x00SpikeInr\x03\x01\x00\x00h\xbbX\x0f\x00\x00\x00Fragment_Lengthr\x04\x01\x00\x00G@~\x00\x00\x00\x00\x00\x00X\x0b\x00\x00\x00Fragment_SDr\x05\x01\x00\x00h\xbbX\r\x00\x00\x00Concentrationr\x06\x01\x00\x00h\xbbX\x07\x00\x00\x00260/280r\x07\x01\x00\x00h\xbbX\x07\x00\x00\x00260/230r\x08\x01\x00\x00h\xbbX\x0b\x00\x00\x00Lab_Commentr\t\x01\x00\x00h\xbbuX\x01\x00\x00\x004r\n\x01\x00\x00}r\x0b\x01\x00\x00(X\x02\x00\x00\x00R1r\x0c\x01\x00\x00X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/4/4_S4_L001_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/4/4_S4_L002_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/4/4_S4_L003_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/4/4_S4_L004_R1_001.fastq.gzr\r\x01\x00\x00X\x02\x00\x00\x00R2r\x0e\x01\x00\x00X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/4/4_S4_L001_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/4/4_S4_L002_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/4/4_S4_L003_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/4/4_S4_L004_R2_001.fastq.gzr\x0f\x01\x00\x00X\n\x00\x00\x00paired_endr\x10\x01\x00\x00K\x01X\t\x00\x00\x00Sample_IDr\x11\x01\x00\x00j\n\x01\x00\x00X\x0b\x00\x00\x00External_IDr\x12\x01\x00\x00h\xbbX\n\x00\x00\x00Project_IDr\x13\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832r\x14\x01\x00\x00X\x08\x00\x00\x00Organismr\x15\x01\x00\x00h\xbbX\x10\x00\x00\x00Sample_Biosourcer\x16\x01\x00\x00h\xbbX[\x00\x00\x00Index1_p7 (If dual indexed libraries are submitted indicate what index sequence is used P7)r\x17\x01\x00\x00X\t\x00\x00\x00SI-GA-B12r\x18\x01\x00\x00XP\x00\x00\x00Index2_p5 (If libraries are submitted  indicate what index sequence is used P75)r\x19\x01\x00\x00X\t\x00\x00\x00SI-GA-B12r\x1a\x01\x00\x00X\x0c\x00\x00\x00Sample_Groupr\x1b\x01\x00\x00h\xbbX\x10\x00\x00\x00Customer_Commentr\x1c\x01\x00\x00h\xbbX\n\x00\x00\x00Project IDr\x1d\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832r\x1e\x01\x00\x00X\x03\x00\x00\x00RINr\x1f\x01\x00\x00h\xbbX\x07\x00\x00\x00SpikeInr \x01\x00\x00h\xbbX\x0f\x00\x00\x00Fragment_Lengthr!\x01\x00\x00G@~\x00\x00\x00\x00\x00\x00X\x0b\x00\x00\x00Fragment_SDr"\x01\x00\x00h\xbbX\r\x00\x00\x00Concentrationr#\x01\x00\x00h\xbbX\x07\x00\x00\x00260/280r$\x01\x00\x00h\xbbX\x07\x00\x00\x00260/230r%\x01\x00\x00h\xbbX\x0b\x00\x00\x00Lab_Commentr&\x01\x00\x00h\xbbuX\x01\x00\x00\x005r\'\x01\x00\x00}r(\x01\x00\x00(X\x02\x00\x00\x00R1r)\x01\x00\x00X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/5/5_S5_L001_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/5/5_S5_L002_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/5/5_S5_L003_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/5/5_S5_L004_R1_001.fastq.gzr*\x01\x00\x00X\x02\x00\x00\x00R2r+\x01\x00\x00X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/5/5_S5_L001_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/5/5_S5_L002_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/5/5_S5_L003_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/5/5_S5_L004_R2_001.fastq.gzr,\x01\x00\x00X\n\x00\x00\x00paired_endr-\x01\x00\x00K\x01X\t\x00\x00\x00Sample_IDr.\x01\x00\x00j\'\x01\x00\x00X\x0b\x00\x00\x00External_IDr/\x01\x00\x00h\xbbX\n\x00\x00\x00Project_IDr0\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832r1\x01\x00\x00X\x08\x00\x00\x00Organismr2\x01\x00\x00h\xbbX\x10\x00\x00\x00Sample_Biosourcer3\x01\x00\x00h\xbbX[\x00\x00\x00Index1_p7 (If dual indexed libraries are submitted indicate what index sequence is used P7)r4\x01\x00\x00X\x08\x00\x00\x00SI-GA-C1r5\x01\x00\x00XP\x00\x00\x00Index2_p5 (If libraries are submitted  indicate what index sequence is used P75)r6\x01\x00\x00X\x08\x00\x00\x00SI-GA-C1r7\x01\x00\x00X\x0c\x00\x00\x00Sample_Groupr8\x01\x00\x00h\xbbX\x10\x00\x00\x00Customer_Commentr9\x01\x00\x00h\xbbX\n\x00\x00\x00Project IDr:\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832r;\x01\x00\x00X\x03\x00\x00\x00RINr<\x01\x00\x00h\xbbX\x07\x00\x00\x00SpikeInr=\x01\x00\x00h\xbbX\x0f\x00\x00\x00Fragment_Lengthr>\x01\x00\x00G@~\x00\x00\x00\x00\x00\x00X\x0b\x00\x00\x00Fragment_SDr?\x01\x00\x00h\xbbX\r\x00\x00\x00Concentrationr@\x01\x00\x00h\xbbX\x07\x00\x00\x00260/280rA\x01\x00\x00h\xbbX\x07\x00\x00\x00260/230rB\x01\x00\x00h\xbbX\x0b\x00\x00\x00Lab_CommentrC\x01\x00\x00h\xbbuX\x01\x00\x00\x006rD\x01\x00\x00}rE\x01\x00\x00(X\x02\x00\x00\x00R1rF\x01\x00\x00X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/6/6_S6_L001_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/6/6_S6_L002_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/6/6_S6_L003_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/6/6_S6_L004_R1_001.fastq.gzrG\x01\x00\x00X\x02\x00\x00\x00R2rH\x01\x00\x00X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/6/6_S6_L001_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/6/6_S6_L002_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/6/6_S6_L003_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/6/6_S6_L004_R2_001.fastq.gzrI\x01\x00\x00X\n\x00\x00\x00paired_endrJ\x01\x00\x00K\x01X\t\x00\x00\x00Sample_IDrK\x01\x00\x00jD\x01\x00\x00X\x0b\x00\x00\x00External_IDrL\x01\x00\x00h\xbbX\n\x00\x00\x00Project_IDrM\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832rN\x01\x00\x00X\x08\x00\x00\x00OrganismrO\x01\x00\x00h\xbbX\x10\x00\x00\x00Sample_BiosourcerP\x01\x00\x00h\xbbX[\x00\x00\x00Index1_p7 (If dual indexed libraries are submitted indicate what index sequence is used P7)rQ\x01\x00\x00X\x08\x00\x00\x00SI-GA-C2rR\x01\x00\x00XP\x00\x00\x00Index2_p5 (If libraries are submitted  indicate what index sequence is used P75)rS\x01\x00\x00X\x08\x00\x00\x00SI-GA-C2rT\x01\x00\x00X\x0c\x00\x00\x00Sample_GrouprU\x01\x00\x00h\xbbX\x10\x00\x00\x00Customer_CommentrV\x01\x00\x00h\xbbX\n\x00\x00\x00Project IDrW\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832rX\x01\x00\x00X\x03\x00\x00\x00RINrY\x01\x00\x00h\xbbX\x07\x00\x00\x00SpikeInrZ\x01\x00\x00h\xbbX\x0f\x00\x00\x00Fragment_Lengthr[\x01\x00\x00G@~\x00\x00\x00\x00\x00\x00X\x0b\x00\x00\x00Fragment_SDr\\\x01\x00\x00h\xbbX\r\x00\x00\x00Concentrationr]\x01\x00\x00h\xbbX\x07\x00\x00\x00260/280r^\x01\x00\x00h\xbbX\x07\x00\x00\x00260/230r_\x01\x00\x00h\xbbX\x0b\x00\x00\x00Lab_Commentr`\x01\x00\x00h\xbbuX\x01\x00\x00\x007ra\x01\x00\x00}rb\x01\x00\x00(X\x02\x00\x00\x00R1rc\x01\x00\x00X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/7/7_S7_L001_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/7/7_S7_L002_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/7/7_S7_L003_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/7/7_S7_L004_R1_001.fastq.gzrd\x01\x00\x00X\x02\x00\x00\x00R2re\x01\x00\x00X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/7/7_S7_L001_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/7/7_S7_L002_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/7/7_S7_L003_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/7/7_S7_L004_R2_001.fastq.gzrf\x01\x00\x00X\n\x00\x00\x00paired_endrg\x01\x00\x00K\x01X\t\x00\x00\x00Sample_IDrh\x01\x00\x00ja\x01\x00\x00X\x0b\x00\x00\x00External_IDri\x01\x00\x00h\xbbX\n\x00\x00\x00Project_IDrj\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832rk\x01\x00\x00X\x08\x00\x00\x00Organismrl\x01\x00\x00h\xbbX\x10\x00\x00\x00Sample_Biosourcerm\x01\x00\x00h\xbbX[\x00\x00\x00Index1_p7 (If dual indexed libraries are submitted indicate what index sequence is used P7)rn\x01\x00\x00X\x08\x00\x00\x00SI-GA-C3ro\x01\x00\x00XP\x00\x00\x00Index2_p5 (If libraries are submitted  indicate what index sequence is used P75)rp\x01\x00\x00X\x08\x00\x00\x00SI-GA-C3rq\x01\x00\x00X\x0c\x00\x00\x00Sample_Grouprr\x01\x00\x00h\xbbX\x10\x00\x00\x00Customer_Commentrs\x01\x00\x00h\xbbX\n\x00\x00\x00Project IDrt\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832ru\x01\x00\x00X\x03\x00\x00\x00RINrv\x01\x00\x00h\xbbX\x07\x00\x00\x00SpikeInrw\x01\x00\x00h\xbbX\x0f\x00\x00\x00Fragment_Lengthrx\x01\x00\x00G@~\x00\x00\x00\x00\x00\x00X\x0b\x00\x00\x00Fragment_SDry\x01\x00\x00h\xbbX\r\x00\x00\x00Concentrationrz\x01\x00\x00h\xbbX\x07\x00\x00\x00260/280r{\x01\x00\x00h\xbbX\x07\x00\x00\x00260/230r|\x01\x00\x00h\xbbX\x0b\x00\x00\x00Lab_Commentr}\x01\x00\x00h\xbbuX\x01\x00\x00\x008r~\x01\x00\x00}r\x7f\x01\x00\x00(X\x02\x00\x00\x00R1r\x80\x01\x00\x00X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/8/8_S8_L001_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/8/8_S8_L002_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/8/8_S8_L003_R1_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/8/8_S8_L004_R1_001.fastq.gzr\x81\x01\x00\x00X\x02\x00\x00\x00R2r\x82\x01\x00\x00X#\x01\x00\x00211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/8/8_S8_L001_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/8/8_S8_L002_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/8/8_S8_L003_R2_001.fastq.gz,211118_NB501038_0317_AH5HMTBGXK/GCF-2021-832/8/8_S8_L004_R2_001.fastq.gzr\x83\x01\x00\x00X\n\x00\x00\x00paired_endr\x84\x01\x00\x00K\x01X\t\x00\x00\x00Sample_IDr\x85\x01\x00\x00j~\x01\x00\x00X\x0b\x00\x00\x00External_IDr\x86\x01\x00\x00h\xbbX\n\x00\x00\x00Project_IDr\x87\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832r\x88\x01\x00\x00X\x08\x00\x00\x00Organismr\x89\x01\x00\x00h\xbbX\x10\x00\x00\x00Sample_Biosourcer\x8a\x01\x00\x00h\xbbX[\x00\x00\x00Index1_p7 (If dual indexed libraries are submitted indicate what index sequence is used P7)r\x8b\x01\x00\x00X\x08\x00\x00\x00SI-GA-C4r\x8c\x01\x00\x00XP\x00\x00\x00Index2_p5 (If libraries are submitted  indicate what index sequence is used P75)r\x8d\x01\x00\x00X\x08\x00\x00\x00SI-GA-C4r\x8e\x01\x00\x00X\x0c\x00\x00\x00Sample_Groupr\x8f\x01\x00\x00h\xbbX\x10\x00\x00\x00Customer_Commentr\x90\x01\x00\x00h\xbbX\n\x00\x00\x00Project IDr\x91\x01\x00\x00X\x0c\x00\x00\x00GCF-2021-832r\x92\x01\x00\x00X\x03\x00\x00\x00RINr\x93\x01\x00\x00h\xbbX\x07\x00\x00\x00SpikeInr\x94\x01\x00\x00h\xbbX\x0f\x00\x00\x00Fragment_Lengthr\x95\x01\x00\x00G@~\x00\x00\x00\x00\x00\x00X\x0b\x00\x00\x00Fragment_SDr\x96\x01\x00\x00h\xbbX\r\x00\x00\x00Concentrationr\x97\x01\x00\x00h\xbbX\x07\x00\x00\x00260/280r\x98\x01\x00\x00h\xbbX\x07\x00\x00\x00260/230r\x99\x01\x00\x00h\xbbX\x0b\x00\x00\x00Lab_Commentr\x9a\x01\x00\x00h\xbbuuX\x02\x00\x00\x00dbr\x9b\x01\x00\x00}r\x9c\x01\x00\x00(X\x08\x00\x00\x00add_erccr\x9d\x01\x00\x00\x89X\x0c\x00\x00\x00reference_dbr\x9e\x01\x00\x00X\x0b\x00\x00\x0010xgenomicsr\x9f\x01\x00\x00X\x0b\x00\x00\x0010xgenomicsr\xa0\x01\x00\x00}r\xa1\x01\x00\x00(X\x07\x00\x00\x00releaser\xa2\x01\x00\x00X\x06\x00\x00\x002020-Ar\xa3\x01\x00\x00X\x08\x00\x00\x00add_erccr\xa4\x01\x00\x00\x89X\x08\x00\x00\x00assemblyr\xa5\x01\x00\x00X\x06\x00\x00\x00GRCm38r\xa6\x01\x00\x00uX\x07\x00\x00\x00ensemblr\xa7\x01\x00\x00}r\xa8\x01\x00\x00(X\x07\x00\x00\x00releaser\xa9\x01\x00\x00X\x03\x00\x00\x00104r\xaa\x01\x00\x00X\r\x00\x00\x00all_divisionsr\xab\x01\x00\x00X2\x00\x00\x00vertebrates,protists,metazoa,plants,fungi,bacteriar\xac\x01\x00\x00X\x08\x00\x00\x00add_erccr\xad\x01\x00\x00\x89j\xa5\x01\x00\x00X\x06\x00\x00\x00GRCm39r\xae\x01\x00\x00X\x08\x00\x00\x00divisionr\xaf\x01\x00\x00X\x0b\x00\x00\x00vertebratesr\xb0\x01\x00\x00uX\x0e\x00\x00\x00ensemblgenomesr\xb1\x01\x00\x00}r\xb2\x01\x00\x00(X\x07\x00\x00\x00releaser\xb3\x01\x00\x00X\x02\x00\x00\x0051r\xb4\x01\x00\x00X\x08\x00\x00\x00add_erccr\xb5\x01\x00\x00\x89uX\x07\x00\x00\x00gencoder\xb6\x01\x00\x00}r\xb7\x01\x00\x00(X\x07\x00\x00\x00releaser\xb8\x01\x00\x00X\x02\x00\x00\x0037r\xb9\x01\x00\x00X\x08\x00\x00\x00add_erccr\xba\x01\x00\x00\x89uX\x06\x00\x00\x00refseqr\xbb\x01\x00\x00}r\xbc\x01\x00\x00(X\x07\x00\x00\x00releaser\xbd\x01\x00\x00X\x03\x00\x00\x00205r\xbe\x01\x00\x00X\x08\x00\x00\x00add_erccr\xbf\x01\x00\x00\x89uX\x07\x00\x00\x00lncbookr\xc0\x01\x00\x00}r\xc1\x01\x00\x00X\x07\x00\x00\x00releaser\xc2\x01\x00\x00X\x04\x00\x00\x009.28r\xc3\x01\x00\x00sX\x07\x00\x00\x00mirbaser\xc4\x01\x00\x00}r\xc5\x01\x00\x00(X\x07\x00\x00\x00releaser\xc6\x01\x00\x00X\x02\x00\x00\x0022r\xc7\x01\x00\x00X\x0f\x00\x00\x00add_calibratorsr\xc8\x01\x00\x00\x89X\x08\x00\x00\x00add_erccr\xc9\x01\x00\x00\x89uX\x08\x00\x00\x00mircartar\xca\x01\x00\x00}r\xcb\x01\x00\x00(X\x07\x00\x00\x00releaser\xcc\x01\x00\x00X\x03\x00\x00\x001.1r\xcd\x01\x00\x00X\x0f\x00\x00\x00add_calibratorsr\xce\x01\x00\x00\x89X\x08\x00\x00\x00add_erccr\xcf\x01\x00\x00\x89uX\x07\x00\x00\x00pirbaser\xd0\x01\x00\x00}r\xd1\x01\x00\x00X\x07\x00\x00\x00releaser\xd2\x01\x00\x00X\x03\x00\x00\x002.0r\xd3\x01\x00\x00sX\x07\x00\x00\x00pirnadbr\xd4\x01\x00\x00}r\xd5\x01\x00\x00X\x07\x00\x00\x00releaser\xd6\x01\x00\x00X\x05\x00\x00\x001_7_6r\xd7\x01\x00\x00sX\n\x00\x00\x00rnacentralr\xd8\x01\x00\x00}r\xd9\x01\x00\x00(X\x08\x00\x00\x00protocolr\xda\x01\x00\x00X\x04\x00\x00\x00httpr\xdb\x01\x00\x00X\x07\x00\x00\x00releaser\xdc\x01\x00\x00X\x04\x00\x00\x0014.0r\xdd\x01\x00\x00uX\x06\x00\x00\x00unitasr\xde\x01\x00\x00}r\xdf\x01\x00\x00X\x07\x00\x00\x00releaser\xe0\x01\x00\x00X\x06\x00\x00\x00210412r\xe1\x01\x00\x00sX\x05\x00\x00\x00uniter\xe2\x01\x00\x00}r\xe3\x01\x00\x00(X\x07\x00\x00\x00releaser\xe4\x01\x00\x00X\x03\x00\x00\x008_2r\xe5\x01\x00\x00X\x05\x00\x00\x00levelr\xe6\x01\x00\x00X\x07\x00\x00\x00dynamicr\xe7\x01\x00\x00uX\x05\x00\x00\x00silvar\xe8\x01\x00\x00}r\xe9\x01\x00\x00(X\x07\x00\x00\x00releaser\xea\x01\x00\x00X\x03\x00\x00\x00132r\xeb\x01\x00\x00X\x05\x00\x00\x00levelr\xec\x01\x00\x00X\x02\x00\x00\x0099r\xed\x01\x00\x00uX\n\x00\x00\x00greengenesr\xee\x01\x00\x00}r\xef\x01\x00\x00(X\x07\x00\x00\x00releaser\xf0\x01\x00\x00X\x04\x00\x00\x0013_8r\xf1\x01\x00\x00X\x05\x00\x00\x00levelr\xf2\x01\x00\x00X\x02\x00\x00\x0097r\xf3\x01\x00\x00uX\x06\x00\x00\x00frnadbr\xf4\x01\x00\x00}r\xf5\x01\x00\x00X\x07\x00\x00\x00releaser\xf6\x01\x00\x00X\x08\x00\x00\x0020160329r\xf7\x01\x00\x00sX\t\x00\x00\x00lncipediar\xf8\x01\x00\x00}r\xf9\x01\x00\x00X\x07\x00\x00\x00releaser\xfa\x01\x00\x00X\x03\x00\x00\x005_2r\xfb\x01\x00\x00sX\t\x00\x00\x00sortmernar\xfc\x01\x00\x00}r\xfd\x01\x00\x00X\x02\x00\x00\x00dbr\xfe\x01\x00\x00X\x9f\x00\x00\x00silva-arc-16s-id95,silva-arc-23s-id98,silva-bac-16s-id90,silva-bac-23s-id98,silva-euk-18s-id95,silva-euk-28s-id98,rfam-5.8s-database-id98,rfam-5s-database-id98r\xff\x01\x00\x00sj\xa5\x01\x00\x00j\xa6\x01\x00\x00uX\x06\x00\x00\x00filterr\x00\x02\x00\x00}r\x01\x02\x00\x00(X\x17\x00\x00\x00compress_filtered_fastqr\x02\x02\x00\x00\x89X\x05\x00\x00\x00dedupr\x03\x02\x00\x00\x89X\x04\x00\x00\x00trimr\x04\x02\x00\x00}r\x05\x02\x00\x00(X\x06\x00\x00\x00filterr\x06\x02\x00\x00\x89X\n\x00\x00\x00quantifierr\x07\x02\x00\x00X\x05\x00\x00\x00fastpr\x08\x02\x00\x00X\x05\x00\x00\x00fastpr\t\x02\x00\x00}r\n\x02\x00\x00(X\x06\x00\x00\x00paramsr\x0b\x02\x00\x00X\x05\x00\x00\x00-l 28r\x0c\x02\x00\x00X\t\x00\x00\x00trim_argsr\r\x02\x00\x00h\xbbuX\x08\x00\x00\x00mirtracer\x0e\x02\x00\x00}r\x0f\x02\x00\x00X\x08\x00\x00\x00protocolr\x10\x02\x00\x00NsuX\x07\x00\x00\x00spikeinr\x11\x02\x00\x00}r\x12\x02\x00\x00(X\x06\x00\x00\x00filterr\x13\x02\x00\x00\x89X\n\x00\x00\x00quantifierr\x14\x02\x00\x00X\x04\x00\x00\x00skipr\x15\x02\x00\x00X\x03\x00\x00\x00refr\x16\x02\x00\x00X\x06\x00\x00\x00ERCC92r\x17\x02\x00\x00uX\t\x00\x00\x00ribosomalr\x18\x02\x00\x00}r\x19\x02\x00\x00(X\x06\x00\x00\x00filterr\x1a\x02\x00\x00\x89X\n\x00\x00\x00quantifierr\x1b\x02\x00\x00X\x04\x00\x00\x00skipr\x1c\x02\x00\x00X\x03\x00\x00\x00refr\x1d\x02\x00\x00X\x04\x00\x00\x00ncbir\x1e\x02\x00\x00uX\x0c\x00\x00\x00contaminantsr\x1f\x02\x00\x00}r \x02\x00\x00(X\x06\x00\x00\x00filterr!\x02\x00\x00\x89X\n\x00\x00\x00quantifierr"\x02\x00\x00X\x04\x00\x00\x00skipr#\x02\x00\x00X\x03\x00\x00\x00refr$\x02\x00\x00X\x0b\x00\x00\x00univec_corer%\x02\x00\x00uuX\x08\x00\x00\x00analysisr&\x02\x00\x00}r\'\x02\x00\x00X\x02\x00\x00\x00qcr(\x02\x00\x00}r)\x02\x00\x00X\x03\x00\x00\x00bfqr*\x02\x00\x00}r+\x02\x00\x00X\x06\x00\x00\x00strainr,\x02\x00\x00NX\x10\x00\x00\x00read_orientationr-\x02\x00\x00X\x07\x00\x00\x00reverser.\x02\x00\x00X\t\x00\x00\x00sample_idr/\x02\x00\x00X\t\x00\x00\x00Sample_IDr0\x02\x00\x00X\x0f\x00\x00\x00sample_group_idr1\x02\x00\x00X\x0c\x00\x00\x00Sample_Groupr2\x02\x00\x00X\x08\x00\x00\x00batch_idr3\x02\x00\x00NX\x08\x00\x00\x00block_idr4\x02\x00\x00NX\x05\x00\x00\x00alignr5\x02\x00\x00}r6\x02\x00\x00(X\r\x00\x00\x00transcriptomer7\x02\x00\x00}r8\x02\x00\x00X\x07\x00\x00\x00alignerr9\x02\x00\x00X\x03\x00\x00\x00bwar:\x02\x00\x00sX\x06\x00\x00\x00genomer;\x02\x00\x00}r<\x02\x00\x00X\x07\x00\x00\x00alignerr=\x02\x00\x00X\x04\x00\x00\x00starr>\x02\x00\x00suX\x08\x00\x00\x00assemblyr?\x02\x00\x00}r@\x02\x00\x00(X\x0c\x00\x00\x00base_ref_dirrA\x02\x00\x00X\x07\x00\x00\x00ensemblrB\x02\x00\x00X\r\x00\x00\x00transcriptomerC\x02\x00\x00}rD\x02\x00\x00(X\t\x00\x00\x00assemblerrE\x02\x00\x00X\x04\x00\x00\x00skiprF\x02\x00\x00X\x05\x00\x00\x00mergerG\x02\x00\x00}rH\x02\x00\x00X\x04\x00\x00\x00typerI\x02\x00\x00X\x05\x00\x00\x00fastqrJ\x02\x00\x00suX\x06\x00\x00\x00genomerK\x02\x00\x00}rL\x02\x00\x00X\t\x00\x00\x00assemblerrM\x02\x00\x00X\x04\x00\x00\x00skiprN\x02\x00\x00suX\x07\x00\x00\x00multiqcrO\x02\x00\x00}rP\x02\x00\x00(X\x06\x00\x00\x00rnaseqrQ\x02\x00\x00}rR\x02\x00\x00X\x07\x00\x00\x00modulesrS\x02\x00\x00X;\x00\x00\x00fastq_screen,star,picard,fastp,fastqc_rnaseq,custom_contentrT\x02\x00\x00sX\n\x00\x00\x00microbiomerU\x02\x00\x00}rV\x02\x00\x00X\x07\x00\x00\x00modulesrW\x02\x00\x00X6\x00\x00\x00fastq_screen,fastp,fastqc_rnaseq,custom_content,qiime2rX\x02\x00\x00sX\n\x00\x00\x00singlecellrY\x02\x00\x00}rZ\x02\x00\x00X\x07\x00\x00\x00modulesr[\x02\x00\x00XT\x00\x00\x00fastq_screen,cellranger,starsolo,fastp,fastqc_rnaseq,custom_content,cellranger_countr\\\x02\x00\x00sX\x08\x00\x00\x00smallrnar]\x02\x00\x00}r^\x02\x00\x00X\x07\x00\x00\x00modulesr_\x02\x00\x00XB\x00\x00\x00fastq_screen,star,picard,fastp,fastqc_rnaseq,unitas,custom_contentr`\x02\x00\x00sX\x07\x00\x00\x00defaultra\x02\x00\x00}rb\x02\x00\x00X\x07\x00\x00\x00modulesrc\x02\x00\x00X/\x00\x00\x00fastq_screen,fastp,fastqc_rnaseq,custom_contentrd\x02\x00\x00sX\x03\x00\x00\x00wgsre\x02\x00\x00}rf\x02\x00\x00X\x07\x00\x00\x00modulesrg\x02\x00\x00X0\x00\x00\x00fastq_screen,fastqc_rnaseq,qualimap,fastp,picardrh\x02\x00\x00suX\x05\x00\x00\x00proxyri\x02\x00\x00}rj\x02\x00\x00(X\x04\x00\x00\x00wgetrk\x02\x00\x00X\x97\x00\x00\x00-e use_proxy=yes -e http_proxy=gcf-proxy.medisin.ntnu.no:8080 -e https_proxy=gcf-proxy.medisin.ntnu.no:8080 -e ftp_proxy=gcf-proxy.medisin.ntnu.no:8080rl\x02\x00\x00X\x05\x00\x00\x00rsyncrm\x02\x00\x00h\xbbX\x04\x00\x00\x00curlrn\x02\x00\x00X\'\x00\x00\x00--proxy gcf-proxy.medisin.ntnu.no:8080 ro\x02\x00\x00X\x06\x00\x00\x00serverrp\x02\x00\x00}rq\x02\x00\x00(X\x04\x00\x00\x00httprr\x02\x00\x00X%\x00\x00\x00http://gcf-proxy.medisin.ntnu.no:8080rs\x02\x00\x00X\x05\x00\x00\x00httpsrt\x02\x00\x00X%\x00\x00\x00http://gcf-proxy.medisin.ntnu.no:8080ru\x02\x00\x00X\x03\x00\x00\x00ftprv\x02\x00\x00X%\x00\x00\x00http://gcf-proxy.medisin.ntnu.no:8080rw\x02\x00\x00uuX\n\x00\x00\x00winecellarrx\x02\x00\x00}ry\x02\x00\x00X\x03\x00\x00\x00urlrz\x02\x00\x00X&\x00\x00\x00https://gcf-winecellar.medisin.ntnu.nor{\x02\x00\x00sX\x0c\x00\x00\x00libprep_namer|\x02\x00\x00XE\x00\x00\x0010X_Genomics_Chromium_Single_Cell_3p_GEM_Library_&_Gel_Bead_Kit_v3_PEr}\x02\x00\x00X\x08\x00\x00\x00workflowr~\x02\x00\x00X\n\x00\x00\x00singlecellr\x7f\x02\x00\x00X\x06\x00\x00\x00dockerr\x80\x02\x00\x00}r\x81\x02\x00\x00(X\x07\x00\x00\x00defaultr\x82\x02\x00\x00X \x00\x00\x00gcfntnu/python-base-conda:4.10.3r\x83\x02\x00\x00X\x04\x00\x00\x00ivarr\x84\x02\x00\x00X\x12\x00\x00\x00gcfntnu/ivar:1.3.1r\x85\x02\x00\x00X\x0c\x00\x00\x00bwa_samtoolsr\x86\x02\x00\x00X\x1b\x00\x00\x00gcfntnu/bwa_samtools:0.7.17r\x87\x02\x00\x00X\x08\x00\x00\x00bedtoolsr\x88\x02\x00\x00X"\x00\x00\x00biocontainers/bedtools:v2.28.0_cv2r\x89\x02\x00\x00X\n\x00\x00\x00primerclipr\x8a\x02\x00\x00X\x18\x00\x00\x00gcfntnu/primerclip:0.3.8r\x8b\x02\x00\x00X\x05\x00\x00\x00fastvr\x8c\x02\x00\x00X\x13\x00\x00\x00gcfntnu/fastv:0.8.1r\x8d\x02\x00\x00X\t\x00\x00\x00bioseqzipr\x8e\x02\x00\x00X\x18\x00\x00\x00gcfntnu/bioseqzip:latestr\x8f\x02\x00\x00X\x06\x00\x00\x00fastqcr\x90\x02\x00\x00X \x00\x00\x00biocontainers/fastqc:v0.11.9_cv7r\x91\x02\x00\x00X\x0c\x00\x00\x00fastq-screenr\x92\x02\x00\x00X2\x00\x00\x00quay.io/biocontainers/fastq-screen:0.14.0--pl526_0r\x93\x02\x00\x00X\x07\x00\x00\x00multiqcr\x94\x02\x00\x00X\x1a\x00\x00\x00gcfntnu/multiqc:1.9dev-gcfr\x95\x02\x00\x00X\x06\x00\x00\x00hisat2r\x96\x02\x00\x00X2\x00\x00\x00quay.io/biocontainers/hisat2:2.1.0--py36h2d50403_1r\x97\x02\x00\x00X\x06\x00\x00\x00bowtier\x98\x02\x00\x00X\x1f\x00\x00\x00biocontainers/bowtie:v1.1.2_cv4r\x99\x02\x00\x00X\x0f\x00\x00\x00bowtie_samtoolsr\x9a\x02\x00\x00X\x1d\x00\x00\x00gcfntnu/bowtie_samtools:1.2.3r\x9b\x02\x00\x00X\x07\x00\x00\x00bowtie2r\x9c\x02\x00\x00X(\x00\x00\x00biocontainers/bowtie2:v2.3.4.3-1-deb_cv1r\x9d\x02\x00\x00X\x10\x00\x00\x00bowtie2_samtoolsr\x9e\x02\x00\x00X\x1e\x00\x00\x00gcfntnu/bowtie2_samtools:2.3.5r\x9f\x02\x00\x00X\x05\x00\x00\x00bbmapr\xa0\x02\x00\x00X-\x00\x00\x00quay.io/biocontainers/bbmap:38.75--h516909a_0r\xa1\x02\x00\x00X\x04\x00\x00\x00starr\xa2\x02\x00\x00X\x13\x00\x00\x00gcfntnu/star:2.7.9ar\xa3\x02\x00\x00X\n\x00\x00\x00quikmirseqr\xa4\x02\x00\x00X\x17\x00\x00\x00gcfntnu/quickmirseq:1.0r\xa5\x02\x00\x00X\x08\x00\x00\x00tximportr\xa6\x02\x00\x00X\x17\x00\x00\x00gcfntnu/tximport:1.18.0r\xa7\x02\x00\x00X\n\x00\x00\x00cellrangerr\xa8\x02\x00\x00X\x18\x00\x00\x00gcfntnu/cellranger:6.0.2r\xa9\x02\x00\x00X\x05\x00\x00\x00mirger\xaa\x02\x00\x00X\x11\x00\x00\x00gcfntnu/mirge:2.0r\xab\x02\x00\x00X\x06\x00\x00\x00unitasr\xac\x02\x00\x00X\x14\x00\x00\x00gcfntnu/unitas:1.7.5r\xad\x02\x00\x00X\x08\x00\x00\x00sambambar\xae\x02\x00\x00X\x14\x00\x00\x00gcfntnu/sambamba:0.7r\xaf\x02\x00\x00X\x06\x00\x00\x00salmonr\xb0\x02\x00\x00X\x14\x00\x00\x00gcfntnu/salmon:1.5.2r\xb1\x02\x00\x00X\x07\x00\x00\x00subreadr\xb2\x02\x00\x00X/\x00\x00\x00quay.io/biocontainers/subread:1.6.3--h84994c4_1r\xb3\x02\x00\x00X\x05\x00\x00\x00rseqcr\xb4\x02\x00\x00X1\x00\x00\x00quay.io/biocontainers/rseqc:3.0.0--py36hf8a1672_0r\xb5\x02\x00\x00X\x06\x00\x00\x00preseqr\xb6\x02\x00\x00X.\x00\x00\x00quay.io/biocontainers/preseq:2.0.3--hf53bd2b_3r\xb7\x02\x00\x00X\x08\x00\x00\x00samtoolsr\xb8\x02\x00\x00X\x12\x00\x00\x00kfdrc/samtools:1.9r\xb9\x02\x00\x00X\x0b\x00\x00\x00picard_gatkr\xba\x02\x00\x00X\x15\x00\x00\x00gcfntnu/picard:2.21.5r\xbb\x02\x00\x00X\x05\x00\x00\x00qortsr\xbc\x02\x00\x00X)\x00\x00\x00quay.io/biocontainers/qorts:1.3.0--py35_0r\xbd\x02\x00\x00X\x06\x00\x00\x00bedopsr\xbe\x02\x00\x00X\x13\x00\x00\x00kfdrc/bedops:2.4.36r\xbf\x02\x00\x00X\x0c\x00\x00\x00ucsc-scriptsr\xc0\x02\x00\x00X\x1b\x00\x00\x00gcfntnu/ucsc-scripts:latestr\xc1\x02\x00\x00X\x05\x00\x00\x00fastpr\xc2\x02\x00\x00X\x1f\x00\x00\x00biocontainers/fastp:v0.20.1_cv1r\xc3\x02\x00\x00X\x08\x00\x00\x00qualimapr\xc4\x02\x00\x00X(\x00\x00\x00quay.io/biocontainers/qualimap:2.2.2d--1r\xc5\x02\x00\x00X\x08\x00\x00\x00mirtracer\xc6\x02\x00\x00X\x16\x00\x00\x00gcfntnu/mirtrace:1.0.1r\xc7\x02\x00\x00X\t\x00\x00\x00stringtier\xc8\x02\x00\x00X/\x00\x00\x00quay.io/biocontainers/stringtie:2.0--hc900ff6_0r\xc9\x02\x00\x00X\x04\x00\x00\x00tacor\xca\x02\x00\x00X\x12\x00\x00\x00gcfntnu/taco:0.7.3r\xcb\x02\x00\x00X\x07\x00\x00\x00scallopr\xcc\x02\x00\x00X\x16\x00\x00\x00gcfntnu/scallop:0.10.4r\xcd\x02\x00\x00X\t\x00\x00\x00sambamabar\xce\x02\x00\x00X0\x00\x00\x00quay.io/biocontainers/sambamba:0.7.1--h148d290_0r\xcf\x02\x00\x00X\n\x00\x00\x00gffcomparer\xd0\x02\x00\x00X3\x00\x00\x00quay.io/biocontainers/gffcompare:0.11.2--h6bb024c_0r\xd1\x02\x00\x00X\x07\x00\x00\x00gffreadr\xd2\x02\x00\x00X0\x00\x00\x00quay.io/biocontainers/gffread:0.11.6--h8b12597_0r\xd3\x02\x00\x00X\t\x00\x00\x00pyensemblr\xd4\x02\x00\x00X+\x00\x00\x00quay.io/biocontainers/pyensembl:1.8.4--py_0r\xd5\x02\x00\x00X\x04\x00\x00\x00rsemr\xd6\x02\x00\x00X1\x00\x00\x00quay.io/biocontainers/rsem:1.3.2--pl526ha52163a_1r\xd7\x02\x00\x00X\r\x00\x00\x00entrez-directr\xd8\x02\x00\x00X\x1a\x00\x00\x00gcfntnu/entrez_direct:11.0r\xd9\x02\x00\x00X\x0b\x00\x00\x00quickmirseqr\xda\x02\x00\x00X\x17\x00\x00\x00gcfntnu/quickmirseq:1.0r\xdb\x02\x00\x00X\t\x00\x00\x00sortmernar\xdc\x02\x00\x00X0\x00\x00\x00quay.io/biocontainers/sortmerna:2.1b--he860b03_4r\xdd\x02\x00\x00X\x08\x00\x00\x00sartoolsr\xde\x02\x00\x00X\x16\x00\x00\x00gcfntnu/sartools:1.7.3r\xdf\x02\x00\x00X\x07\x00\x00\x00atroposr\xe0\x02\x00\x00X\x16\x00\x00\x00jdidion/atropos:latestr\xe1\x02\x00\x00X\r\x00\x00\x00fastx_toolkitr\xe2\x02\x00\x00X6\x00\x00\x00quay.io/biocontainers/fastx_toolkit:0.0.14--he1b5a44_8r\xe3\x02\x00\x00X\x08\x00\x00\x00bfq_plotr\xe4\x02\x00\x00X\x14\x00\x00\x00gcfntnu/bfq_plot:0.1r\xe5\x02\x00\x00X\x06\x00\x00\x00qiime2r\xe6\x02\x00\x00X\x16\x00\x00\x00gcfntnu/qiime2:2019.10r\xe7\x02\x00\x00X\n\x00\x00\x00fastq_pairr\xe8\x02\x00\x00X\x16\x00\x00\x00gcfntnu/fastq_pair:0.3r\xe9\x02\x00\x00X\x06\x00\x00\x00scanpyr\xea\x02\x00\x00X\x14\x00\x00\x00gcfntnu/scanpy:1.7.0r\xeb\x02\x00\x00X\x06\x00\x00\x00seuratr\xec\x02\x00\x00X\x18\x00\x00\x00gcfntnu/seurat:dev-3.0.1r\xed\x02\x00\x00X\t\x00\x00\x00umi_toolsr\xee\x02\x00\x00X5\x00\x00\x00quay.io/biocontainers/umi_tools:1.1.1--py38h0213d0e_1r\xef\x02\x00\x00X\x08\x00\x00\x00velocytor\xf0\x02\x00\x00X\x18\x00\x00\x00gcfntnu/velocyto:0.17.17r\xf1\x02\x00\x00X\x0b\x00\x00\x00cellbrowserr\xf2\x02\x00\x00X\x1a\x00\x00\x00gcfntnu/cellbrowser:latestr\xf3\x02\x00\x00X\x08\x00\x00\x00phyloseqr\xf4\x02\x00\x00X\x17\x00\x00\x00gcfntnu/phyloseq:1.28.0r\xf5\x02\x00\x00X\x0e\x00\x00\x00jupyter-scanpyr\xf6\x02\x00\x00X)\x00\x00\x00gcfntnu/scanpy-jupyterlab:1.7.0-snakemaker\xf7\x02\x00\x00X\x06\x00\x00\x00seqkitr\xf8\x02\x00\x00X\x14\x00\x00\x00pegi3s/seqkit:0.12.1r\xf9\x02\x00\x00X\x08\x00\x00\x00cutadaptr\xfa\x02\x00\x00X\x14\x00\x00\x00gcfntnu/cutadapt:3.3r\xfb\x02\x00\x00X\x12\x00\x00\x00jupyter-diff-exprsr\xfc\x02\x00\x00X!\x00\x00\x00gcfntnu/jupyterlab-diff-exprs:1.2r\xfd\x02\x00\x00X\x10\x00\x00\x00jupyter-co-exprsr\xfe\x02\x00\x00X\x1f\x00\x00\x00gcfntnu/jupyterlab-co-exprs:1.0r\xff\x02\x00\x00X\x0e\x00\x00\x00jupyter-rnaseqr\x00\x03\x00\x00X\x1d\x00\x00\x00gcfntnu/jupyterlab-rnaseq:1.3r\x01\x03\x00\x00X\x07\x00\x00\x00gcf-bior\x02\x03\x00\x00X\x13\x00\x00\x00gcfntnu/gcf-bio:1.0r\x03\x03\x00\x00X\t\x00\x00\x00bcl2fastqr\x04\x03\x00\x00X\x18\x00\x00\x00gcfntnu/bcl2fastq:2.20.0r\x05\x03\x00\x00X\x0f\x00\x00\x00cellranger-atacr\x06\x03\x00\x00X\x1d\x00\x00\x00gcfntnu/cellranger-atac:2.0.0r\x07\x03\x00\x00X\x0b\x00\x00\x00spacerangerr\x08\x03\x00\x00X\x19\x00\x00\x00gcfntnu/spaceranger:1.3.0r\t\x03\x00\x00uX\x0c\x00\x00\x00base_ref_dirr\n\x03\x00\x00XI\x00\x00\x00/mnt/archive/ext_cache/ext/10xgenomics/release-2020-A/mus_musculus/GRCm38r\x0b\x03\x00\x00uX\x04\x00\x00\x00ruler\x0c\x03\x00\x00X\x1a\x00\x00\x00cellranger_scanpy_pp_ipynbr\r\x03\x00\x00X\x0f\x00\x00\x00bench_iterationr\x0e\x03\x00\x00NX\t\x00\x00\x00scriptdirr\x0f\x03\x00\x00XV\x00\x00\x00/mnt/bfq-analysis/GCF-2021-832_211118/src/gcf-workflows/singlecell/rules/quant/scriptsr\x10\x03\x00\x00ub.'); from snakemake.logging import logger; logger.printshellcmds = True; import os; os.chdir('/mnt/bfq-analysis/GCF-2021-832_211118');
######## snakemake preamble end #########


In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd

In [ ]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures
sc.logging.print_version_and_date()
sc.set_figure_params(figsize=(12,12))

In [ ]:
adata = sc.read_h5ad(snakemake.input[0])

In [ ]:
sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_genes(adata, min_cells=5)

In [ ]:
data = adata

In [ ]:
data.var['mt'] = data.var["gene_symbols"].str.lower().str.startswith('mt').astype(bool)  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(data, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(data, ['n_genes', 'n_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(data, x='n_counts', y='pct_counts_mt')
sc.pl.scatter(data, x='n_counts', y='n_genes')

In [ ]:
data = data[data.obs['pct_counts_mt'] < 15,:]
data = data[data.obs['n_counts'] > 1e3,:]
data = data[data.obs['n_counts'] < 25e3,:]
sc.pl.scatter(data, x='n_counts', y='pct_counts_mt')
sc.pl.scatter(data, x='n_counts', y='n_genes')

In [ ]:
sc.pl.violin(data, ['n_genes', 'n_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True, save="_post_filter.pdf")

In [ ]:
data

In [ ]:
data.obs["n_counts"].mean()

In [ ]:
sc.pp.normalize_per_cell(data, counts_per_cell_after=1e6)
sc.pl.highest_expr_genes(data, n_top=10, gene_symbols='gene_symbols')

In [ ]:
sc.pp.log1p(data)

In [ ]:
sc.pp.highly_variable_genes(data, n_top_genes=3000)

In [ ]:
sc.pl.highly_variable_genes(data)

In [ ]:
sc.tl.pca(data)

In [ ]:
sc.pl.pca_scatter(data, color=['n_counts', 'n_genes','pct_counts_mt'])

In [ ]:
sc.pp.neighbors(data, n_pcs = 30)
sc.tl.louvain(data)
sc.tl.umap(data)

In [ ]:
sc.pl.umap(data, color=['n_counts', 'n_genes','pct_counts_mt', 'sample_id'], s=50)

In [ ]:
sc.pl.umap(data, color=['louvain'], s=50)

In [ ]:
import scvelo as scv

In [ ]:
scv.pp.moments(data, n_pcs=30, n_neighbors=30)

In [ ]:
scv.tl.velocity(data)

In [ ]:
scv.tl.velocity_graph(data)

In [ ]:
scv.pl.velocity_embedding_stream(data, basis='umap')

In [ ]:
data.write(snakemake.output['preprocessed'])